In [ ]:
# Open camera files
#
# GL Grap Labview VI was for camera framegrabber was modified to store data as binary file.
# UNIT8 binary data, dumped in a file without header etc. Frame by frame. See code below to read frames.
# staff-groups/Diamond/Programs, Code and Drivers/Labview/HL Grab with AVI export function/HL Grab Export Binary  (Express).vi
#   
# Code below reads file, frame by frame.
#
# Previously I stored labview files as AVI with Y800 grayscale codec (NI Vision). 
# However, Python OpenCV could not open these files properly.
# This should normally work: http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html
# Multiple people report errors with Y800 codec and opencv. Could also be ffmpeg problem though.
#
# Wouter, 29 March 2017


import matplotlib.pyplot as plt
import os.path
import numpy as np
%matplotlib inline

filename = '/Users/wjwesterveld/Documents/Temp/20170329/142446_movie.bin'
# filename = '/Users/wjwesterveld/Documents/Temp/20170329/110126_movie.bin' 

# 111700_movie, 111515_movie, 111419_movie, 111238_movie, 111156_movie, 110826_movie, 110742_movie, 110309_movie, 110126_movie

# Read corresponding .txt file into dictionary
fileproperties = {}
with open( filename[0:-4] + '.txt', 'r') as f:
    for line in f:
        (key, val) = line.split("=", 1)        
        fileproperties[key.strip()] = val.strip()

nx = int( fileproperties['nx'] )
ny = int( fileproperties['ny'] )
framerate = float( fileproperties['framerate [Hz]'] )





In [ ]:
# Read movie, first frame
print os.path.isfile(filename)
fo = open(filename,'r')
gray = np.fromfile(fo, dtype=np.uint8, count=nx*ny)
gray = gray.reshape(ny,nx)  
fo.close()
gray2 = gray

In [ ]:
# Analyze frame, play with settings
if False:
    from scipy import ndimage
    gray = gray2

    # plot histogram
    gray = gray2
    plt.hist(gray.reshape((-1,1)), bins=25)  # plt.hist passes it's arguments to np.histogram
    plt.title("Images histograms")
    plt.show()

    # analyze frame
    threshold = 160
    gray = 255 - gray # invert image
    gray = np.clip(gray,threshold,255) - threshold

    # center of mass
    position = ndimage.measurements.center_of_mass(gray)

    # plot
    print position
    plt.figure(figsize=(12, 8))
    plt.imshow(gray,cmap='gray')
    plt.plot( position[1], position[0], 'r+' )
    plt.show()



In [ ]:
# Read movie and analyze motion
from scipy import ndimage

threshold = 160

print 'File exists: ' + str( os.path.isfile(filename) )

x = np.empty( (0) )
y = np.empty( (0) )
firstframe = True
fo = open(filename,'r')
while True:
    # Load frame
    gray = np.fromfile(fo, dtype=np.uint8, count=nx*ny)
    if not gray.size:
        break
    gray = gray.reshape(ny,nx)  
    
    # analyze frame    
    gray = 255 - gray # invert image
    gray = np.clip(gray,threshold,255) - threshold # clip
    position = ndimage.measurements.center_of_mass(gray)
    x = np.append( x, position[1] )
    y = np.append( y, position[0] )
    
    if firstframe:
        firstframe = False
        plt.figure(figsize=(12, 8))
        plt.imshow(gray,cmap='gray')
        plt.plot( position[1], position[0], 'r+' )
        plt.show()

fo.close()



print 'Finished :-)'

In [ ]:
t = np.arange( len(x) ) / framerate

xMicronPerPixel = 38.0 / 353.0
yMicronPerPixel = 37.0 / 430.0

xMicron = x * xMicronPerPixel
yMicron = y * yMicronPerPixel

print 'framerate [Hz]:' + str( framerate )

%matplotlib inline

plt.figure(figsize=(12, 8))
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [\mum]' )
plt.plot( t, xMicron, label='x' )
plt.plot( t, yMicron, label='y' )
plt.legend()
plt.grid()
plt.show()

# x from 5.0 to 12.1 = 7.1 um --> agrees with piezo (7mm)
# y from 7.0 to 11.2 = 4.2 um (first step) --> agrees with piezo movement (4 mm)


if True:
    xi1 = int( 60 * framerate )
    xi2 = int( 80 * framerate )
    xMicron = xMicron[xi1:xi2]
    yMicron = yMicron[xi1:xi2]
    t = t[xi1:xi2]
    
    %matplotlib osx
    plt.figure(figsize=(12, 8))
    plt.xlabel( 'time [s]' )
    plt.ylabel( 'distance [um]' )
    plt.plot( t, xMicron, label='x' )
    plt.legend()
    plt.grid()
    plt.show()        
    
    print 'Results for 60 s to 80 s'

print 'file: ' + filename
print 'title: ' + fileproperties['title']
print 'x std: %.3f um, 3*std: %.3f um, max-min: %.3f um' % ( np.std( xMicron ),  3 * np.std( xMicron ), np.max(xMicron) - np.min(xMicron) )
print 'y std: %.3f um, 3*std: %.3f um, max-min: %.3f um' % ( np.std( yMicron ),  3 * np.std( yMicron ), np.max(yMicron) - np.min(yMicron) )





In [ ]:
from scipy.fftpack import fft
from scipy.signal import blackman
%matplotlib inline

N = len( xMicron )
frequency = np.linspace(0.0, 0.5 * framerate, N//2)

w = blackman(N)
xMicronFFT = fft( (xMicron - np.mean( xMicron) ) * w )
yMicronFFT = fft( (yMicron - np.mean( yMicron) ) * w )

plt.figure(figsize=(12, 8))
plt.plot( frequency, 2.0/N * np.abs( xMicronFFT[0:N//2] ), label='x' )
plt.plot( frequency, 2.0/N * np.abs( yMicronFFT[0:N//2] ), label='y' )
plt.title( 'spectrum. signal filtered with blackman window and fft. ')
plt.xlabel( 'frequency [Hz]' )
plt.ylabel( 'amplitude' ) 
plt.legend()
plt.grid()
plt.show()




In [ ]:

plt.plot((xMicron - np.mean( xMicron) ) *  w )

Results

file/Users/wjwesterveld/Documents/Temp/20170329/111700_movie.bin
title: dirt zoom-in, closed loop piezo, no business card.
x std: 0.034 um, 3*std: 0.103 um, max-min: 0.214 um
y std: 0.051 um, 3*std: 0.154 um, max-min: 0.324 um

file: /Users/wjwesterveld/Documents/Temp/20170329/111515_movie.bin
title: dirt zoom-in, closed loop piezo, no business card.
x std: 0.040 um, 3*std: 0.121 um, max-min: 0.274 um
y std: 0.062 um, 3*std: 0.185 um, max-min: 0.357 um

file: /Users/wjwesterveld/Documents/Temp/20170329/111419_movie.bin
title: dirt zoom-in, closed loop piezo, no business card.
x std: 0.035 um, 3*std: 0.106 um, max-min: 0.205 um
y std: 0.063 um, 3*std: 0.189 um, max-min: 0.385 um


file: /Users/wjwesterveld/Documents/Temp/20170329/111238_movie.bin
title: dirt zoom-in, closed loop piezo, small business card.
x std: 0.042 um, 3*std: 0.125 um, max-min: 0.251 um
y std: 0.051 um, 3*std: 0.152 um, max-min: 0.343 um

file: /Users/wjwesterveld/Documents/Temp/20170329/111156_movie.bin
title: dirt zoom-in, closed loop piezo, small business card.
x std: 0.040 um, 3*std: 0.120 um, max-min: 0.264 um
y std: 0.055 um, 3*std: 0.165 um, max-min: 0.369 um

file: /Users/wjwesterveld/Documents/Temp/20170329/110826_movie.bin
title: dirt zoom-in, open loop piezo, small business card.
x std: 0.040 um, 3*std: 0.121 um, max-min: 0.257 um
y std: 0.053 um, 3*std: 0.158 um, max-min: 0.332 um

file: /Users/wjwesterveld/Documents/Temp/20170329/110742_movie.bin
title: dirt zoom-in, open loop piezo, small business card.
x std: 0.037 um, 3*std: 0.112 um, max-min: 0.238 um
y std: 0.053 um, 3*std: 0.158 um, max-min: 0.333 um

file: /Users/wjwesterveld/Documents/Temp/20170329/110309_movie.bin
title: dirt zoom-in, open loop piezo, no business card.
x std: 0.037 um, 3*std: 0.110 um, max-min: 0.217 um
y std: 0.063 um, 3*std: 0.189 um, max-min: 0.377 um

file: /Users/wjwesterveld/Documents/Temp/20170329/110126_movie.bin
title: dirt zoom-in, open loop piezo, no business card.
x std: 0.037 um, 3*std: 0.111 um, max-min: 0.230 um
y std: 0.062 um, 3*std: 0.187 um, max-min: 0.409 um


Results for 60 s to 80 s
file: /Users/wjwesterveld/Documents/Temp/20170329/142446_movie.bin
title: moving piece of dirt around. x = 177 to 184 um (python); y = 27 to 31 um (python).
x std: 0.048 um, 3*std: 0.143 um, max-min: 0.252 um
y std: 0.080 um, 3*std: 0.241 um, max-min: 0.503 um

